In [1]:
import itertools
import ipywidgets as widgets

import numpy as np
from ipycanvas import Canvas, MultiCanvas

import plotly.graph_objects as go
from plotly.subplots import make_subplots

Some parameters that can be tweaked (carefully, though)

In [2]:
width, height = 900, 600 # in pixels
# width, height = 100, 600 # in pixels

# these pixels are encompassed by those above
horizontal_padding = 300 # in pixels
vertical_padding = 100

# the smaller, the easier the sensors get activated
obs_to_angle_sensitivity = 0.25

tx_power = 2e3

sensors_fill_color = 'red'
position_fill_color = 'green'

n_sensors = 4

# sensors are represented as circles
sensors_radius = 19

# maximum value of the power shown in the plots
max_y = 60

# Model

We compute the observation recorded at a sensor placed at position $\mathbf{s}$ when the *target* is at $\mathbf{x}$  as
$$
    y
    =
    \frac{
        P_0
    }
    {
        \left|\left|
            \mathbf{x} - \mathbf{s}
        \right|\right|_2
    }
$$
where $\left|\left|\mathbf{x}\right|\right|_2$ is the modulus (2-norm) of vector $\mathbf{x}$.

In [3]:
def obs(x_1: int, y_1: int, x_sensor: int, y_sensor) -> float:
    """
        Computes an observation given the x and y coordinates for two different positions.
    """
    
    return tx_power / np.sqrt((x_1 - x_sensor)**2 + (y_1 - y_sensor)**2)

The functions going from the model observation to the angle in the *canvas*' meters is
$$
2\pi
\left(
\frac{
    1
}{
    1 + e^{-\rho y}
}
- 0.5
\right)
/
(1-0.5)
$$
where $\rho$ is the above *observation-to-angle sensitivity* (`obs_to_angle_sensitivity`).

In [4]:
def obs_to_angle(obs: float) -> float:
    """
        Turns a measure into an angle in radians.
    """
    
    return 2*np.pi * ((1. / (1. + np.exp(-obs_to_angle_sensitivity*obs))) - 0.5) / (1 - 0.5)

# Preliminary computations

In [5]:
# number of sensors per axis
n_sensors_per_axis = int(np.sqrt(n_sensors))

# all the x coordinates of the sensors...
sensors_x_coordinates = np.linspace(horizontal_padding, width-horizontal_padding, n_sensors_per_axis)
# sensors_x_coordinates = np.linspace(horizontal_padding, width-horizontal_padding, n_sensors_per_axis+)[1:]

# ...and all the y's
sensors_y_coordinates = np.linspace(vertical_padding, height-vertical_padding, n_sensors_per_axis)

# a list of tuples with the resulting positions
sensors_coordinates = list(itertools.product(sensors_x_coordinates, sensors_y_coordinates))

n_sensors = len(sensors_coordinates)

*Global* variables are used to keep track of the clicked positions.

In [6]:
target_position = []
observations = []

# Canvas

We instantiate a 2-layers canvas: one for the sensors (fixed), and one for the (user-recorded) positions.

In [7]:
canvas = MultiCanvas(n_canvases=2, width=width, height=height)

## Background

The filling style and color for the background canvas ($\#0$) are set up.

In [8]:
canvas[0].fill_style = sensors_fill_color
canvas[0].stroke_style = 'blue'

# Border

The region in which the target can move.

In [9]:
canvas[0].stroke_rect(0, 0, canvas[0].width, canvas[0].height)

# Sensors

Sensors are plotted.

In [10]:
# for every sensor's x-y coordinates
for x, y in sensors_coordinates:
    
    # the sensor is plotted (outlined)
    canvas[0].stroke_arc(x, y, 20, 0, 2*np.pi)

A container for the *plotly* figures.

In [11]:
# Create FigureWidget for real-time updates
fig_widget = go.FigureWidget(make_subplots(
    rows=n_sensors_per_axis, cols=n_sensors_per_axis,
    subplot_titles=[f"Sensor {i+1}" for i in range(n_sensors)]
))

A *plotly* figure for every sensor.

In [14]:
# Create scatter plots for each sensor
for i in range(n_sensors):
    # row, col = divmod(i, n_sensors_per_axis)
    col, row = divmod(i, n_sensors_per_axis)
    trace = go.Scatter(x=[], y=[], mode='lines', name=f'Sensor {i+1}')
    fig_widget.add_trace(trace, row=row+1, col=col+1)
    fig_widget.update_yaxes(range=[0, max_y], row=row+1, col=col+1)

Some global tweaks.

In [15]:
# Update layout
fig_widget.update_layout(
    height=600, width=800,
    title_text="Observations",
    showlegend=False,
);

## Events handlers

Handler for when mouse is *un-clicked*

In [16]:
def handle_mouse_up(x, y):
    print(f"Mouse clicked at: ({x}, {y})")

    target_position.append((x, y))
    canvas[1].fill_style = position_fill_color
    canvas[1].fill_rect(x, y, 10, 10)

    new_observations = [None] * n_sensors
    for i_sensor, (x_sensor, y_sensor) in enumerate(sensors_coordinates):
        canvas[1].fill_style = 'white'
        canvas[1].fill_arc(x_sensor, y_sensor, sensors_radius, 0, 2*np.pi)

        new_observations[i_sensor] = obs(x, y, x_sensor, y_sensor)
        
        canvas[1].fill_style = sensors_fill_color
        canvas[1].fill_arc(x_sensor, y_sensor, sensors_radius, 0, obs_to_angle(new_observations[i_sensor]))

    observations.append(new_observations)
    observations_np = np.array(observations)
    x_vals = np.arange(observations_np.shape[0])

    with fig_widget.batch_update():
        for i, trace in enumerate(fig_widget.data):
            trace.x = x_vals
            trace.y = observations_np[:, i]

canvas[1].on_mouse_up(handle_mouse_up)


Next cell provides the canvas. By clicking any point on the *map* we record the position of the target at a (discrete) time instant. This has two effects:
- Each sensor fills up to the extent of its received signal strength (starting from $0$ degrees and going clock-wise).
- The plots above are updated with the signal seen so far by every sensor (you need to click more than once for this to be shown).

It is probably a good idea to **zoom out** a little bit to see the canvas and the plots simultaneously.

In [17]:
# Display everything
display(fig_widget)
canvas

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Sensor 1',
              'type': 'scatter',
              'uid': '9d8d73ee-8257-4641-91a2-6fafba641217',
              'x': [],
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'mode': 'lines',
              'name': 'Sensor 2',
              'type': 'scatter',
              'uid': '3f56b205-489a-49ec-a4c2-3c2cddb5e015',
              'x': [],
              'xaxis': 'x3',
              'y': [],
              'yaxis': 'y3'},
             {'mode': 'lines',
              'name': 'Sensor 3',
              'type': 'scatter',
              'uid': 'c77e40fe-299b-4bbe-956f-80b16c39c58b',
              'x': [],
              'xaxis': 'x2',
              'y': [],
              'yaxis': 'y2'},
             {'mode': 'lines',
              'name': 'Sensor 4',
              'type': 'scatter',
              'uid': 'afef75b0-f27a-4b5a-ac09-464cc606e764',
              'x': [],
             

MultiCanvas(height=600, width=900)